In [3]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
import altair as alt

In [6]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Re-load the dataset specifying that it does not contain a header
df = pd.read_csv("FowlerModule07-1.csv", header=None)

# Rename columns 0 to `Year` and 1 to `Attendance`, with column index starting from 0
df = df.rename(columns={0: "Year", 1: "Attendance"})

# Drop the first row containing the redundant header
df = df.drop(df.index[0])

# Convert `Year` and `Attendance` to numeric
df["Year"] = pd.to_numeric(df["Year"])
df["Attendance"] = pd.to_numeric(df["Attendance"])

# Calculate the 3-period moving average
df["MA_3"] = df["Attendance"].rolling(window=3).mean()

# Calculate the exponential smoothing forecast with a smoothing constant of 0.7
df["ES_0.7"] = df["Attendance"].ewm(alpha=0.7, adjust=False).mean()

# Calculate MAD, MSE, and MAPE for `MA_3`
df['MAD_MA_3'] = abs(df['Attendance'] - df['MA_3'])
df['MSE_MA_3'] = (df['Attendance'] - df['MA_3']) ** 2
df['MAPE_MA_3'] = abs((df['Attendance'] - df['MA_3']) / df['Attendance'])

# Calculate MAD, MSE, and MAPE for `ES_0.7`
df['MAD_ES_0.7'] = abs(df['Attendance'] - df['ES_0.7'])
df['MSE_ES_0.7'] = (df['Attendance'] - df['ES_0.7']) ** 2
df['MAPE_ES_0.7'] = abs((df['Attendance'] - df['ES_0.7']) / df['Attendance'])

# Drop the first two rows containing NaN values for `MA_3`
df = df.dropna()

# Print the descriptive statistics of the new columns
print("Descriptive Statistics of MAD, MSE, and MAPE for MA_3:")
print(df[['MAD_MA_3', 'MSE_MA_3', 'MAPE_MA_3']].describe().to_markdown(numalign="left", stralign="left"))

print("\nDescriptive Statistics of MAD, MSE, and MAPE for ES_0.7:")
print(df[['MAD_ES_0.7', 'MSE_ES_0.7', 'MAPE_ES_0.7']].describe().to_markdown(numalign="left", stralign="left"))

# Create a multiple line series plot over time.
chart = (
    alt.Chart(
        # Melt by `Year` so that the different columns come under the same stacked column. Rename the columns accordingly.
        pd.melt(
            df,
            "Year",
            var_name="Attendance Type",
            value_name="Attendance Value",
            value_vars=["Attendance", "MA_3", "ES_0.7"],
        )
    )
    .mark_line()
    .encode(
        # Set `Year` on the x-axis.
        x=alt.X("Year", axis=alt.Axis(format="%Y")),
        # Set `Attendance Value` on the y-axis and add appropriate title to the axis
        y=alt.Y("Attendance Value"),
        # Have a different color for each `Attendance Type`
        color="Attendance Type",
        # Add tooltips for the relevant features to show details on hover
        tooltip=[
            alt.Tooltip("Year", format="%Y"),
            "Attendance Value",
            "Attendance Type",
        ],
    )
    .properties(title="Attendance, MA_3, and ES_0.7 Over Time")
    .interactive()
)  # Add interactive features for zoom and pan

# Save the chart in a JSON file
chart.save("attendance_ma3_es07_over_time.json")

Descriptive Statistics of MAD, MSE, and MAPE for MA_3:
|       | MAD_MA_3   | MSE_MA_3   | MAPE_MA_3   |
|:------|:-----------|:-----------|:------------|
| count | 19         | 19         | 19          |
| mean  | 149        | 44258.5    | 0.0904451   |
| std   | 152.587    | 93815.2    | 0.0781681   |
| min   | 0          | 0          | 0           |
| 25%   | 51.1667    | 2618.72    | 0.0340494   |
| 50%   | 129.333    | 16727.1    | 0.0782889   |
| 75%   | 214.833    | 46237.4    | 0.15382     |
| max   | 643.333    | 413878     | 0.283782    |

Descriptive Statistics of MAD, MSE, and MAPE for ES_0.7:
|       | MAD_ES_0.7   | MSE_ES_0.7   | MAPE_ES_0.7   |
|:------|:-------------|:-------------|:--------------|
| count | 19           | 19           | 19            |
| mean  | 75.9037      | 10229.7      | 0.0461797     |
| std   | 68.6772      | 20002.4      | 0.0350453     |
| min   | 4.05537      | 16.446       | 0.00258139    |
| 25%   | 33.1203      | 1096.99      | 0.0209664  

In [7]:
# Rename columns 0 to `Year` and 1 to `Attendance`, with column index starting from 0
df = df.rename(columns={0: "Year", 1: "Attendance"})

# Drop the first row containing the redundant header
df = df.drop(df.index[0])

# Convert `Year` and `Attendance` to numeric
df["Year"] = pd.to_numeric(df["Year"])
df["Attendance"] = pd.to_numeric(df["Attendance"])

# Define the range of moving average periods and smoothing constants
ma_periods = range(2, 6)
alphas = [i / 10 for i in range(3, 10)]

# Store the results
results = []

# Calculate and evaluate moving average models
for period in ma_periods:
    df[f'MA_{period}'] = df['Attendance'].rolling(window=period).mean()
    mad = mean_absolute_error(df['Attendance'].iloc[period:], df[f'MA_{period}'].iloc[period:])
    mse = mean_squared_error(df['Attendance'].iloc[period:], df[f'MA_{period}'].iloc[period:])
    mape = (
        abs((df['Attendance'].iloc[period:] - df[f'MA_{period}'].iloc[period:]) / df['Attendance'].iloc[period:])
    ).mean() * 100
    results.append(['Moving Average', period, mad, mse, mape])

# Calculate and evaluate exponential smoothing models
for alpha in alphas:
    df[f'ES_{alpha}'] = df['Attendance'].ewm(alpha=alpha, adjust=False).mean()
    mad = mean_absolute_error(df['Attendance'].iloc[1:], df[f'ES_{alpha}'].iloc[1:])
    mse = mean_squared_error(df['Attendance'].iloc[1:], df[f'ES_{alpha}'].iloc[1:])
    mape = (
        abs((df['Attendance'].iloc[1:] - df[f'ES_{alpha}'].iloc[1:]) / df['Attendance'].iloc[1:])
    ).mean() * 100
    results.append(['Exponential Smoothing', alpha, mad, mse, mape])

# Create a DataFrame from the results
results_df = pd.DataFrame(results, columns=['Method', 'Parameter', 'MAD', 'MSE', 'MAPE'])

# Print the results
print(results_df.to_markdown(index=False, numalign="left", stralign="left"))

| Method                | Parameter   | MAD     | MSE     | MAPE    |
|:----------------------|:------------|:--------|:--------|:--------|
| Moving Average        | 2           | 145.125 | 37930.7 | 8.76282 |
| Moving Average        | 3           | 155.667 | 48540.7 | 9.42993 |
| Moving Average        | 4           | 174.143 | 57570.3 | 10.6028 |
| Moving Average        | 5           | 212.369 | 75467.5 | 12.9377 |
| Exponential Smoothing | 0.3         | 153.606 | 42429.3 | 9.3964  |
| Exponential Smoothing | 0.4         | 133.938 | 33276.7 | 8.17754 |
| Exponential Smoothing | 0.5         | 115.258 | 24725   | 7.02086 |
| Exponential Smoothing | 0.6         | 95.5062 | 16953.2 | 5.80453 |
| Exponential Smoothing | 0.7         | 74.3791 | 10243.2 | 4.5121  |
| Exponential Smoothing | 0.8         | 51.5853 | 4910.21 | 3.12481 |
| Exponential Smoothing | 0.9         | 26.8842 | 1331.02 | 1.62673 |


A moving average with a parameter of 2 has the lowest error for MAPE, MAD, and MSE.  The smoothing 0.9 has the lowest error for all three as well.